In [29]:
import pandas as pd
import requests
import gzip
import io
from sqlalchemy import create_engine
import psycopg2
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import pickle

In [3]:
# Connection en local de la base de donnée (en brute car par manque de temps pas eu le temps de sécuriser les connections)

username = 'postgres'   
password = 'root'   
hostname = 'localhost'   
port     = '5432'  
database = 'postgres'
sslmode  = 'require'  

def connectEnginePostgresql(username,password,hostname,db,ssl=None,port=None):
    conn_string = f"host={hostname} dbname={db} user={username} password={password}"
    return conn_string


# Connection string for PostgreSQL
connection_string = connectEnginePostgresql(username,password,hostname,database)
conn              = psycopg2.connect(connection_string)
cursor            = conn.cursor()

In [4]:
# Commande SQL pour récupérer les données de nos features et target, on ne choisis que les données qui ont un type "Movie"
cursor.execute('''select tb.tconst, tb."primaryTitle", tb."titleType", tb.genres, 
tb."startYear", tb."runtimeMinutes", 
array_to_string(cbt."actor",','),
array_to_string(cbt."director",','),
array_to_string(cbt."writer",','),
array_to_string(cbt."producer",','),
array_to_string(cbt."cinematographer",','),
array_to_string(cbt."composer",','),
array_to_string(cbt."editor",','),
array_to_string(cbt."production_designer",','),
array_to_string(cbt."self",','),
array_to_string(cbt."archive_footage",','),
array_to_string(cbt."archive_sound",','),
tr."averageRating", tr."numVotes",
 count(ta.title)
from "titleBasics" tb 
full outer join "titleAkas" ta 
on tb.tconst = ta."titleId" 
full outer join cast_by_title cbt 
on tb.tconst = cbt.tconst 
full outer join "titleRatings" tr 
on tb.tconst  = tr.tconst 
where
tb."isAdult" = 0
AND tb."titleType" LIKE 'movie'
group by tb.tconst,
cbt."actor",cbt."director",
cbt."writer",cbt."producer",
cbt."cinematographer",cbt."composer",
cbt."editor",cbt."production_designer",
cbt."self",cbt."archive_footage",
cbt."archive_sound",
tr."averageRating", tr."numVotes"
ORDER BY tr."numVotes" DESC NULLS LAST, tr."averageRating" DESC NULLS LAST
LIMIT 10000
''' )

# On créé notre dataset 
records = cursor.fetchall()
df = pd.DataFrame(records , 
columns=["tconst","title","titleType","genres","startYear","runtimeMinutes","actor","director","writer","producer",
         "cinematographer","composer","editor","production_designer","self","archive_footage","archive_sound","averageRating","numVotes","countTitleByRegion"])

In [6]:
df

,tconst,title,titleType,genres,startYear,runtimeMinutes,actor,director,writer,producer,cinematographer,composer,editor,production_designer,self,archive_footage,archive_sound,averageRating,numVotes,countTitleByRegion
0,tt0111161,The Shawshank Redemption,movie,Drama,1994,142,"William Sadler,Bob Gunton,Tim Robbins,Morgan F...",Frank Darabont,Stephen King,Niki Marvin,Roger Deakins,Thomas Newman,Richard Francis-Bruce,None,None,None,None,9.3,2851572,82
1,tt0468569,The Dark Knight,movie,"Action,Crime,Drama",2008,152,"Heath Ledger,Christian Bale,Michael Caine,Aaro...",Christopher Nolan,"Bob Kane,Jonathan Nolan,David S. Goyer","Charles Roven,Lorne Orleans",None,None,None,None,None,None,None,9.0,2832886,77
2,tt1375666,Inception,movie,"Action,Adventure,Sci-Fi",2010,148,"Leonardo DiCaprio,Ken Watanabe,Elliot Page,Jos...",Christopher Nolan,None,Emma Thomas,Wally Pfister,Hans Zimmer,Lee Smith,Guy Hendrix Dyas,None,None,None,8.8,2515111,67
3,tt0137523,Fight Club,movie,Drama,1999,139,"Brad Pitt,Edward Norton,Meat Loaf,Zach Grenier",David Fincher,"Jim Uhls,Chuck Palahniuk","Ross Grayson Bell,Art Linson,Ceán Chaffin",None,None,None,None,None,None,None,8.8,2286960,61
4,tt0109830,Forrest Gump,movie,"Drama,Romance",1994,142,"Gary Sinise,Sally Field,Robin Wright,Tom Hanks",Robert Zemeckis,"Winston Groom,Eric Roth","Steve Tisch,Steve Starkey,Wendy Finerman",None,None,None,None,None,None,None,8.8,2224625,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,tt2571140,Boss,movie,"Action,Comedy,Crime",2013,143,"Mithun Chakraborty,Shiv Panditt,Akshay Kumar,R...",Anthony D'Souza,"Udaykrishnan,Sajid,Farhad Samji,Siby K. Thomas",Jyoti Kapur Das,None,None,None,None,None,None,None,5.1,11749,9
9996,tt0061369,Asterix the Gaul,movie,"Action,Adventure,Animation",1967,68,"Roger Carel,Jacques Morel,Pierre Tornade,Jacqu...",Ray Goossens,"Jos Marissen,Willy Lateste,René Goscinny,Alber...",None,None,None,None,None,None,None,None,6.6,11744,43
9997,tt0142242,Dragon Ball Z: Broly - The Legendary Super Saiyan,movie,"Action,Animation,Fantasy",1993,72,"Mayumi Tanaka,Toshio Furukawa,Takeshi Kusao,Ma...",Shigeyasu Yamauchi,"Akira Toriyama,Takao Koyama","Tomio Anzai,Chiaki Imada",None,Shunsuke Kikuchi,None,None,None,None,None,7.4,11738,33
9998,tt0456470,The Bow,movie,"Drama,Romance",2005,90,"Ji-Seok Seo,Han Yeo-reum,Jeon Gook-hwan,Seong-...",Kim Ki-duk,None,None,Seong-back Jang,Eun-il Kang,None,Sol Art Chung,None,None,None,7.1,11738,28


In [7]:
# On regroupe dans une liste chaque rôle 
casting = ["actor","director","writer","producer","cinematographer","composer","editor","production_designer","self","archive_footage","archive_sound"]

In [8]:
# On remplace les NaN par une valeur vide pour les features catégorielle 
for cast in casting:
    df[cast] = df[cast].fillna('')
df["genres"] = df["genres"].str.replace(',',' ')
df['genres'] = df['genres'].fillna('')



#On remplace les NaN par 0 pour les features numérique, une exception pour l'averageRating car c'est notre target donc on remplace par la moyenne 
df['startYear'] = pd.to_numeric(df['startYear'], errors='coerce').fillna(0)
df['runtimeMinutes'] = pd.to_numeric(df['runtimeMinutes'], errors='coerce').fillna(0)
df['countTitleByRegion'] = pd.to_numeric(df['runtimeMinutes'], errors='coerce').fillna(0)
df['numVotes'] = pd.to_numeric(df['runtimeMinutes'], errors='coerce').fillna(0)
df['averageRating'] = pd.to_numeric(df['averageRating'], errors='coerce').fillna(df['averageRating'].mean())

In [9]:
# Cette fonction concat les données par ligne et leur rôle 
def cast(row):
    return 'director: '+" "+row['director']+" "+'actor: '+" "+row["actor"]+" "+'self: '+" "+row["self"]+" "+'producer: '+" "+row["producer"]+" "+'writer: '+" "+row["writer"]+" "+'cinematographer: '+" "+row["cinematographer"]+" "+'composer: '+" "+row["composer"]+" "+'editor: '+" "+row["editor"]+" "+'production_designer: '+" "+row["production_designer"]

In [10]:
# Application de la concaténation dans une colonne "cast"
df["cast"] = df.apply(cast,axis=1)

In [11]:
df.head()

,tconst,title,titleType,genres,startYear,runtimeMinutes,actor,director,writer,producer,...,composer,editor,production_designer,self,archive_footage,archive_sound,averageRating,numVotes,countTitleByRegion,cast
0,tt0111161,The Shawshank Redemption,movie,Drama,1994,142,"William Sadler,Bob Gunton,Tim Robbins,Morgan F...",Frank Darabont,Stephen King,Niki Marvin,...,Thomas Newman,Richard Francis-Bruce,,,,,9.3,2851572,82,director: Frank Darabont actor: William Sadl...
1,tt0468569,The Dark Knight,movie,Action Crime Drama,2008,152,"Heath Ledger,Christian Bale,Michael Caine,Aaro...",Christopher Nolan,"Bob Kane,Jonathan Nolan,David S. Goyer","Charles Roven,Lorne Orleans",...,,,,,,,9.0,2832886,77,director: Christopher Nolan actor: Heath Led...
2,tt1375666,Inception,movie,Action Adventure Sci-Fi,2010,148,"Leonardo DiCaprio,Ken Watanabe,Elliot Page,Jos...",Christopher Nolan,,Emma Thomas,...,Hans Zimmer,Lee Smith,Guy Hendrix Dyas,,,,8.8,2515111,67,director: Christopher Nolan actor: Leonardo ...
3,tt0137523,Fight Club,movie,Drama,1999,139,"Brad Pitt,Edward Norton,Meat Loaf,Zach Grenier",David Fincher,"Jim Uhls,Chuck Palahniuk","Ross Grayson Bell,Art Linson,Ceán Chaffin",...,,,,,,,8.8,2286960,61,"director: David Fincher actor: Brad Pitt,Edw..."
4,tt0109830,Forrest Gump,movie,Drama Romance,1994,142,"Gary Sinise,Sally Field,Robin Wright,Tom Hanks",Robert Zemeckis,"Winston Groom,Eric Roth","Steve Tisch,Steve Starkey,Wendy Finerman",...,,,,,,,8.8,2224625,59,director: Robert Zemeckis actor: Gary Sinise...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,tt2571140,Boss,movie,Action Comedy Crime,2013,143,"Mithun Chakraborty,Shiv Panditt,Akshay Kumar,R...",Anthony D'Souza,"Udaykrishnan,Sajid,Farhad Samji,Siby K. Thomas",Jyoti Kapur Das,...,,,,,,,5.1,11749,9,director: Anthony D'Souza actor: Mithun Chak...
9996,tt0061369,Asterix the Gaul,movie,Action Adventure Animation,1967,68,"Roger Carel,Jacques Morel,Pierre Tornade,Jacqu...",Ray Goossens,"Jos Marissen,Willy Lateste,René Goscinny,Alber...",,...,,,,,,,6.6,11744,43,"director: Ray Goossens actor: Roger Carel,Ja..."
9997,tt0142242,Dragon Ball Z: Broly - The Legendary Super Saiyan,movie,Action Animation Fantasy,1993,72,"Mayumi Tanaka,Toshio Furukawa,Takeshi Kusao,Ma...",Shigeyasu Yamauchi,"Akira Toriyama,Takao Koyama","Tomio Anzai,Chiaki Imada",...,Shunsuke Kikuchi,,,,,,7.4,11738,33,director: Shigeyasu Yamauchi actor: Mayumi T...
9998,tt0456470,The Bow,movie,Drama Romance,2005,90,"Ji-Seok Seo,Han Yeo-reum,Jeon Gook-hwan,Seong-...",Kim Ki-duk,,,...,Eun-il Kang,,Sol Art Chung,,,,7.1,11738,28,"director: Kim Ki-duk actor: Ji-Seok Seo,Han ..."


In [13]:
# Mise en place des features et de la target
X = df.drop(['averageRating',"actor","director","writer","producer","cinematographer","composer","editor","production_designer","self","archive_footage","archive_sound"],axis=1)
y = df['averageRating']  

In [25]:
# Création de la Pipeline : On passe par un column transformer pour vectoriser les données catégorielle et on utilise un scaler pour les données numériques.


transformer = ColumnTransformer(
    transformers=[
        (
        'vectorizerGenre',
         CountVectorizer(),
         'genres'
         ),(
         'vectorizerCast',
         CountVectorizer(),
         'cast'
         ),(
         'vectorizerTitle',
         CountVectorizer(),
         'title'
         ),
         (
        'data_num',
         RobustScaler(),
         ['countTitleByRegion', 'numVotes','startYear','runtimeMinutes']
        )
    ])

# Split du dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creation de la pipeline avec le transformer et le modèle 
pipeline = Pipeline([
    ('transform', transformer), 
    ('regressor', RandomForestRegressor(criterion='absolute_error',n_estimators=100,min_samples_split=6))
])

In [15]:
X_train.head()

,tconst,title,titleType,genres,startYear,runtimeMinutes,numVotes,countTitleByRegion,cast
9254,tt0252591,King of the Doormen,movie,Comedy Drama,1976,84.0,84.0,84.0,"director: Zeki Ökten actor: Sevket Altug,Bil..."
1561,tt0465602,Shoot 'Em Up,movie,Action Comedy Thriller,2007,86.0,86.0,86.0,"director: Michael Davis actor: Clive Owen,Pa..."
1670,tt0120784,Payback,movie,Action Crime Drama,1999,100.0,100.0,100.0,"director: Brian Helgeland actor: Mel Gibson,..."
6087,tt1032751,The Warrior's Way,movie,Action Drama Fantasy,2010,100.0,100.0,100.0,"director: Sngmoo Lee actor: Geoffrey Rush,Da..."
6669,tt0241025,Vanity Fair,movie,Drama,2004,141.0,141.0,141.0,director: Mira Nair actor: Reese Witherspoon...
...,...,...,...,...,...,...,...,...,...
5734,tt0100133,Memphis Belle,movie,Action Drama War,1990,107.0,107.0,107.0,director: Michael Caton-Jones actor: Tate Do...
5191,tt0104454,Howards End,movie,Drama Romance,1992,142.0,142.0,142.0,director: James Ivory actor: Anthony Hopkins...
5390,tt1976000,The Two Faces of January,movie,Crime Mystery Romance,2014,96.0,96.0,96.0,"director: Hossein Amini actor: Daisy Bevan,K..."
860,tt0118655,Austin Powers: International Man of Mystery,movie,Adventure Comedy,1997,94.0,94.0,94.0,"director: Jay Roach actor: Mike Myers,Mimi R..."


In [16]:
y_train.head()

9254    8.4
1561    6.6
1670    7.1
6087    6.2
6669    6.2
       ... 
5734    6.9
5191    7.4
5390    6.2
860     7.0
7270    5.4
Name: averageRating, Length: 8000, dtype: float64

In [17]:
pipeline

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('vectorizerGenre',
                                                  CountVectorizer(), 'genres'),
                                                 ('vectorizerCast',
                                                  CountVectorizer(), 'cast'),
                                                 ('vectorizerTitle',
                                                  CountVectorizer(), 'title'),
                                                 ('data_num', RobustScaler(),
                                                  ['countTitleByRegion',
                                                   'numVotes', 'startYear',
                                                   'runtimeMinutes'])])),
                ('regressor', RandomForestRegressor())])

In [22]:
# Utilisation de RandomizedSearchCV pour trouver les meilleurs paramètre pour le modèle (plus de 40h d'entrainement de modèle pour les trouver !)

'''param = {      
    'regressor__n_estimators':[100],
    'regressor__criterion':['absolute_error'],
    'regressor__min_samples_split':[6],
    'regressor__max_depth' : [None],
    }




grid = RandomizedSearchCV(pipeline,param,n_jobs=-1 ,verbose=2, random_state=42,scoring='neg_mean_absolute_error')
grid.fit(X_train, y_train)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid.best_estimator_)
print("\n The best score across ALL searched params:\n",grid.best_score_)
print("\n The best parameters across ALL searched params:\n",grid.best_params_)'''


Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Grid Search 

 The best estimator across ALL searched params:
 Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('vectorizerGenre',
                                                  CountVectorizer(), 'genres'),
                                                 ('vectorizerCast',
                                                  CountVectorizer(), 'cast'),
                                                 ('vectorizerTitle',
                                                  CountVectorizer(), 'title'),
                                                 ('data_num', RobustScaler(),
                                                  ['countTitleByRegion',
                                                   'numVotes', 'startYear',
                                                   'runtimeMinutes'])])),
                ('regressor',
                 RandomForestRegressor(criterion='absolute

In [26]:
# Entrainement du modèle 
pipeline.fit(X_train, y_train)

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('vectorizerGenre',
                                                  CountVectorizer(), 'genres'),
                                                 ('vectorizerCast',
                                                  CountVectorizer(), 'cast'),
                                                 ('vectorizerTitle',
                                                  CountVectorizer(), 'title'),
                                                 ('data_num', RobustScaler(),
                                                  ['countTitleByRegion',
                                                   'numVotes', 'startYear',
                                                   'runtimeMinutes'])])),
                ('regressor',
                 RandomForestRegressor(criterion='absolute_error',
                                       min_samples_split=6))])

In [27]:
# Sauvegarde de l'entrainement pour s'en servir dans le streamlit 
filename = 'predict_model.sav'
pickle.dump(pipeline, open(filename, 'wb'))

In [30]:
# Mise en place de la prediction.

y_pred = pipeline.predict(X_test)

# Evaluation du modèle et affichage des résultats.
mse = mean_squared_error(y_test, y_pred=y_pred)
mae = mean_absolute_error(y_test,y_pred=y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f'Score du modèle: {pipeline.score(X_test, y_test)}') 

RMSE: 0.8037652008671438
MAE: 0.57065175
Score du modèle: 0.37275669691359137


: 